<a href="https://colab.research.google.com/github/whitechocobread/Ai-project/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyautogui

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.54-py3-none-any.whl size=37576 sha256=7cf0bd1d4081c9d361c394d3742e542546f823bef61ca815db60c

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install konlpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 43.6 MB/s eta 0:00:00


In [7]:
import numpy as np
import pandas as pd
import re
import json

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
file_path = '/content/drive/MyDrive/삼성전자.xlsx'
df = pd.read_excel(file_path, header = 0)

print(df.head())


   labels                            Description 
0      -2         삼성전자 폴더블폰 시장 점유율이 86%에서 72%로 하락
1       1  글로벌 폴더블폰 시장이 전년 대비 16%, 전분기 대비 215% 증가
2      -1         중국 업체들의 공격적인 제품 출시로 삼성전자 점유율 하락
3       2      삼성 갤럭시Z 플립5와 폴드5, 시장에서 가장 잘 팔리는 모델
4       1  4분기 삼성전자 폴더블폰 점유율, 작년 83%에서 42%로 하락 예상


In [22]:
X_df = df['Description ']
y_df = df['labels']
print('본문의 개수: {}'.format(len(X_df)))
print('레이블의 개수: {}'.format(len(y_df)))


본문의 개수: 110
레이블의 개수: 110


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=0, stratify=y_df)
# train_data와 test_data 대신 X_train, X_test 사용
print(f"학습 데이터 크기: {len(X_train)}\n테스트 데이터 크기: {len(X_test)}")

학습 데이터 크기: 88
테스트 데이터 크기: 22


In [29]:
import re
from konlpy.tag import Okt

# 전처리 함수
def preprocessing(review, okt, remove_stopwords=False, stop_words=[]):
    # 한국어가 아닌 문자 제거
    review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', review)

    # 형태소로 토큰화
    word_review = okt.morphs(review_text, stem=True)

    # 필요한 경우 불용어 제거
    if remove_stopwords:
        word_review = [token for token in word_review if token not in stop_words]

    return word_review

# Okt 객체 생성 및 불용어 설정
okt = Okt()
stop_words = ['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한']

# 학습 데이터 전처리
clean_train_review = X_train.apply(lambda x: preprocessing(x, okt, remove_stopwords=True, stop_words=stop_words) if isinstance(x, str) else [])

# 결과 확인
print(clean_train_review[:4])


87          [갤럭시, 방수, 방진, 등급, 먼지, 와, 습기, 걱정, 없다]
11          [직원, 에게, 년, 목표, 설정, 과, 이정표, 세우다, 권장]
67                   [고성능, 컴퓨팅, 시장, 투자, 필요성, 증가]
21    [삼성, 후광, 효과, 로, 용인, 화성, 수원, 지역, 집값, 에, 영향]
Name: Description , dtype: object


In [30]:
# 테스트 데이터에 전처리 함수 적용
clean_test_review = X_test.apply(lambda x: preprocessing(x, okt, remove_stopwords=True, stop_words=stop_words) if isinstance(x, str) else [])

# 결과 확인
print(clean_test_review[:4])


1          [글로벌, 폴, 더블, 폰, 시장, 전, 년, 대비, 전, 분기, 대비, 증가]
16              [직원, 일상, 업무, 에, 너무, 집중, 하다, 말다, 하다, 조언]
19    [하이닉스, 와의, 시장, 경쟁, 에서, 삼, 성, 전자, 수익, 성, 개선, 요원...
46                  [삼성, 전자, 임원, 장기, 성과, 보수, 제도, 운영, 중]
Name: Description , dtype: object


In [31]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# 인덱스 벡터 변환 및 패딩 처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)

word_vocab = tokenizer.word_index # 단어 사전 형태
MAX_SEQUENCE_LENGTH = 8 # 문장 최대 길이

# 학습 데이터
train_inputs = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# 학습 데이터 라벨 벡터화
train_labels = np.array(y_train)

# 평가 데이터
test_inputs = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# 평가 데이터 라벨 벡터화
test_labels = np.array(y_test)


In [47]:
# 학습 데이터 불러오기
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json


#전처리 데이터 불러오기
DATA_PATH = './content/sample_data/CLEAN_DATA/'
DATA_OUT = './content/sample_data/DATA_OUT/'
INPUT_TRAIN_DATA = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

train_input = np.load(open(DATA_PATH + INPUT_TRAIN_DATA,'rb'))
train_input = pad_sequences(train_input,maxlen=train_input.shape[1])
train_label = np.load(open(DATA_PATH + LABEL_TRAIN_DATA,'rb'))
prepro_configs = json.load(open(DATA_PATH+DATA_CONFIGS,'r'))

In [37]:
! pip install matplotlib

In [38]:
model_name= 'cnn_classifier_kr'
BATCH_SIZE = 512
NUM_EPOCHS = 10
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs={'model_name': model_name, 'vocab_size':prepro_configs['vocab_size'],'embbeding_size':128, 'num_filters':100,'dropout_rate':0.5, 'hidden_dimension':250,'output_dimension':1}

In [39]:
class CNNClassifier(tf.keras.Model):

	def __init__(self, **kargs):
		super(CNNClassifier, self).__init__(name=kargs['model_name'])
		self.embedding = layers.Embedding(input_dim=kargs['vocab_size'], output_dim=kargs['embbeding_size'])
		self.conv_list = [layers.Conv1D(filters=kargs['num_filters'], kernel_size=kernel_size, padding='valid',activation = tf.keras.activations.relu,
																		kernel_constraint = tf.keras.constraints.MaxNorm(max_value=3)) for kernel_size in [3,4,5]]
		self.pooling = layers.GlobalMaxPooling1D()
		self.dropout = layers.Dropout(kargs['dropout_rate'])
		self.fc1 = layers.Dense(units=kargs['hidden_dimension'],
														activation = tf.keras.activations.relu,
														kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
		self.fc2 = layers.Dense(units=kargs['output_dimension'],
														activation=tf.keras.activations.sigmoid,
														kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3.))


	def call(self,x):
		x = self.embedding(x)
		x = self.dropout(x)
		x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis = 1)
		x = self.fc1(x)
		x = self.fc2(x)
		return x

In [40]:
from tensorflow.keras.models import save_model

model = CNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy')])

#검증 정확도를 통한 EarlyStopping 기능 및 모델 저장 방식 지정
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)
checkpoint_path = DATA_OUT + model_name +'\weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
  print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
  os.makedirs(checkpoint_dir, exist_ok=True)
  print("{} -- Folder create complete \n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor = 'val_accuracy', verbose=1, save_best_only = True,
    save_weights_only=True
)

history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs = NUM_EPOCHS,
                    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])
# 모델 저장하기
save_model(model,'/model')

./content/sample_data/DATA_OUT -- Folder create complete 

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.6927 - accuracy: 0.1266
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to ./content/sample_data/DATA_OUT/cnn_classifier_kr\weights.h5
1/1 [==============================] - 4s 4s/step - loss: 0.6927 - accuracy: 0.1266 - val_loss: 0.6906 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6771 - accuracy: 0.1646
Epoch 2: val_accuracy improved from 0.00000 to 0.11111, saving model to ./content/sample_data/DATA_OUT/cnn_classifier_kr\weights.h5
1/1 [==============================] - 0s 120ms/step - loss: 0.6771 - accuracy: 0.1646 - val_loss: 0.6970 - val_accuracy: 0.1111
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.6620 - accuracy: 0.2785
Epoch 3: val_accuracy improved from 0.11111 to 0.22222, saving model to ./content/sample_data/DATA_OUT/cnn_classifier_kr\weights.h5
1/1 [========

In [44]:
# 예시: 전처리된 테스트 데이터가 있다고 가정
test_data_preprocessed = ... # 전처리된 테스트 데이터

# 모델 예측
predictions = model.predict(test_data_preprocessed)

# 결과 해석 및 출력
for i, prediction in enumerate(predictions):
    if prediction > 0.5:
        print(f"샘플 {i}: 긍정 ({prediction[0] * 100:.2f}%)")
    else:
        print(f"샘플 {i}: 부정 ({(1 - prediction[0]) * 100:.2f}%)")


TypeError: ignored